In [1]:
import numpy as np
import os
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from tqdm import tqdm
import matplotlib.pyplot as plt
from h5py import File
from scripts.dataset import Dataset
from scripts.model import STRCA, FBTRCA
from scipy.linalg import eigh

In [2]:
if __name__=="__main__":
    Fs = 256
    Co = 11
    num_fold = 5
    num_subj = 15
    max_freq = 10
    step_bank= 2
    frange = np.arange(step_bank,max_freq+1,step_bank)
    channels = (np.int64([1,3,5,15,29,30,31,45,55,57,59])-1).tolist()
    num_bank = int(max_freq/step_bank)
    t  = [0, 2]
    result = np.zeros([num_subj, num_fold, 4])

    for sub in range(num_subj):
        # dataset=Dataset(num_class=7,path='Dataset/DataBank/Dataset I/', channel=channels)
        dataset=Dataset(num_class=7,path='/Volumes/T.C.angle/Dataset/OData/Dataset I/', channel=channels)
        dataset.load_data(sub, None)
        dataset.divide_data(num_fold)
        dataset.filter_data(fs=Fs, frange=frange, order=8, btype='lowpass')
        for fold in tqdm(range(num_fold), desc=f'Sub{sub}:'):
            train_set, train_label, test_set, test_label = dataset.retreive_data(
                    class_index=[0,1,2,3,4,5,6], fold_index=fold, filter_index=np.arange(len(frange))+1)
            train_set = train_set[:,:,int(t[0]*Fs):int(t[1]*Fs),:]
            test_set  = test_set[:,:,int(t[0]*Fs):int(t[1]*Fs),:]

            fbtrca = FBTRCA(Co, frange, optimizer='trca', ifG=False)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 0] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

            fbtrca = FBTRCA(Co, frange, optimizer='trca', ifG=True)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 1] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)
            
            fbtrca = FBTRCA(Co, frange, optimizer='tdlda', ifG=False)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 2] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

            fbtrca = FBTRCA(Co, frange, optimizer='tdlda', ifG=True)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 3] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

Sub14:: 100%|█████████████████████████████████████| 5/5 [06:25<00:00, 77.17s/it]


In [5]:
np.mean(result,axis=(0,1))

array([0.43845173, 0.44258434, 0.44883796, 0.45489843])

In [7]:
if __name__=="__main__":
    Fs = 256
    Co = 11
    num_fold = 5
    num_subj = 15
    max_freq = 10
    step_bank= 2
    frange = np.arange(step_bank,max_freq+1,step_bank)
    channels = (np.int64([1,3,5,15,29,30,31,45,55,57,59])-1).tolist()
    num_bank = int(max_freq/step_bank)
    t  = [0, 2]
    result = np.zeros([num_subj, num_fold, 4])

    for sub in range(num_subj):
        # dataset=Dataset(num_class=7,path='Dataset/DataBank/Dataset I/', channel=channels)
        dataset=Dataset(num_class=7,path='/Volumes/T.C.angle/Dataset/OData/Dataset I/', channel=channels)
        dataset.load_data(sub, None)
        dataset.divide_data(num_fold)
        dataset.filter_data(fs=Fs, frange=frange, order=8, btype='bandpass')
        for fold in tqdm(range(num_fold), desc=f'Sub{sub}:'):
            train_set, train_label, test_set, test_label = dataset.retreive_data(
                    class_index=[0,1,2,3,4,5,6], fold_index=fold, filter_index=np.arange(len(frange))+1)
            train_set = train_set[:,:,int(t[0]*Fs):int(t[1]*Fs),:]
            test_set  = test_set[:,:,int(t[0]*Fs):int(t[1]*Fs),:]

            fbtrca = FBTRCA(Co, frange, optimizer='trca', ifG=False)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 0] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

            fbtrca = FBTRCA(Co, frange, optimizer='trca', ifG=True)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 1] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)
            
            fbtrca = FBTRCA(Co, frange, optimizer='tdlda', ifG=False)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 2] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

            fbtrca = FBTRCA(Co, frange, optimizer='tdlda', ifG=True)
            fbtrca.fit(train_set, train_label)
            train_data = fbtrca.transform(train_set).reshape((len(train_label), fbtrca.K, 8, len(frange)))
            test_data  = fbtrca.transform(test_set).reshape((len(test_label), fbtrca.K, 8, len(frange)))
            svm   = SVC(kernel='rbf')
            svm.fit(train_data.reshape((len(train_label), fbtrca.K*8*len(frange))), train_label)
            result[sub,fold, 3] =svm.score(test_data.reshape((len(test_label), fbtrca.K*8*len(frange))), test_label)

Sub0::   0%|                                              | 0/5 [00:05<?, ?it/s]


KeyboardInterrupt: 

In [5]:
np.mean(result,axis=(0,1))

array([0.40476469, 0.40553569, 0.34420591, 0.34232414])